#DETECT INGREDIENTS FROM BEST.PT AND ITS NUMBERS SEND TO SERVER.JS

In [ ]:
import cv2
from ultralytics import YOLO
import requests
import time

# Load the trained YOLO model
model = YOLO(r"C:\Users\vaish\Desktop\Projects\Recipe_Website\weight\best.pt")  # Update this path

# Start video capture
cap = cv2.VideoCapture(0)

# Set confidence threshold
confidence_threshold = 0.6

# Dictionaries to track detection state
detection_tracker = {}
current_counts = {}  # Tracks the current counts sent to the server
pending_stable_counts = {}  # Tracks pending counts for stability check

# Constants
detection_duration = 4  # Seconds of stable detection before sending to server
removal_duration = 4  # Seconds of absence to consider removal

# Functions to interact with the server
def send_to_server(ingredient, count):
    try:
        ingredient_count = f"{ingredient} x{count}"
        print(f"Sending to server: {ingredient_count}")
        response = requests.post(
            "http://localhost:5000/ingredients",
            json={"ingredients": [ingredient_count]}
        )
        if response.status_code == 200:
            print(f"Successfully sent to server: {ingredient_count}")
        else:
            print(f"Failed to send {ingredient_count}, status code: {response.status_code}")
    except Exception as e:
        print(f"Error sending {ingredient_count}: {e}")

def remove_from_server(ingredient, count):
    try:
        ingredient_count = f"{ingredient} x{count}"
        print(f"Removing from server: {ingredient_count}")
        response = requests.post(
            "http://localhost:5000/remove_ingredient",
            json={"ingredients": [ingredient_count]}
        )
        if response.status_code == 200:
            print(f"Successfully removed from server: {ingredient_count}")
        else:
            print(f"Failed to remove {ingredient_count}, status code: {response.status_code}")
    except Exception as e:
        print(f"Error removing {ingredient_count}: {e}")

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break

    results = model(frame)
    current_time = time.time()

    # Current frame detections
    detected_in_frame = {}

    # Process detections
    for result in results[0].boxes:
        confidence = result.conf.item()
        if confidence >= confidence_threshold:
            ingredient = model.names[int(result.cls.item())]
            detected_in_frame.setdefault(ingredient, 0)
            detected_in_frame[ingredient] += 1

            # Initialize or update stable detection
            if ingredient not in detection_tracker:
                detection_tracker[ingredient] = {"start_time": current_time, "last_seen": current_time, "count": 0}
            else:
                detection_tracker[ingredient]["last_seen"] = current_time
                detection_tracker[ingredient]["count"] = detected_in_frame[ingredient]

    # Handle stable detections
    for ingredient, data in list(detection_tracker.items()):
        count = data["count"]

        if ingredient in detected_in_frame:
            # Check if detection is stable for 4 seconds
            if current_time - data["start_time"] >= detection_duration:
                # If this count is different from the current server count, ensure it is stable
                if ingredient not in current_counts or current_counts[ingredient] != count:
                    # Check for stability
                    if ingredient not in pending_stable_counts:
                        pending_stable_counts[ingredient] = {"count": count, "start_time": current_time}
                    elif (
                        pending_stable_counts[ingredient]["count"] == count and
                        current_time - pending_stable_counts[ingredient]["start_time"] >= detection_duration
                    ):
                        # Stable detection confirmed
                        # Remove old count from server
                        if ingredient in current_counts:
                            remove_from_server(ingredient, current_counts[ingredient])
                        # Send the new count to the server
                        send_to_server(ingredient, count)
                        current_counts[ingredient] = count
                        pending_stable_counts.pop(ingredient)
        else:
            # Remove pending stability check if the ingredient disappears
            if ingredient in pending_stable_counts:
                pending_stable_counts.pop(ingredient)

            # Check if the ingredient should be removed after 4 seconds of absence
            if current_time - data["last_seen"] >= removal_duration:
                if ingredient in current_counts:
                    remove_from_server(ingredient, current_counts[ingredient])
                    current_counts.pop(ingredient)
                detection_tracker.pop(ingredient)

    # Display detections
    for ingredient, count in detected_in_frame.items():
        cv2.putText(frame, f"{ingredient} x{count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow("Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



0: 480x640 2 cheeses, 63.2ms
Speed: 1.0ms preprocess, 63.2ms inference, 130.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 cheeses, 10.7ms
Speed: 3.9ms preprocess, 10.7ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 cheeses, 9.0ms
Speed: 2.3ms preprocess, 9.0ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 cheeses, 12.4ms
Speed: 2.2ms preprocess, 12.4ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 cheeses, 8.8ms
Speed: 3.9ms preprocess, 8.8ms inference, 2.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 cheeses, 11.4ms
Speed: 3.3ms preprocess, 11.4ms inference, 2.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 cheeses, 11.0ms
Speed: 3.4ms preprocess, 11.0ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 cheeses, 11.4ms
Speed: 4.0ms preprocess, 11.4ms inference, 3.9ms postprocess per image at shape (1,